## Loading Pretrained Model

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.2 MB/s eta 0:00:00


In [3]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.9 MB/s eta 0:00:00


In [4]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=3e082177ecaf7e900f9a659d073a17cb12cf861286c8c5eb7a5d4a0e4fa37b30
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [5]:
# loading the libraries
from datasets import Dataset
from datasets import load_dataset
from datasets import load_metric
import torch
from transformers import AutoModelForTokenClassification
from transformers import XLMRobertaTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import gc
import pickle

In [8]:
# loading the encoded datasets
dataset_train = load_dataset("msislam/marc-multilingual-encodings-v4", split = 'train')
dataset_test = load_dataset("msislam/marc-multilingual-encodings-v4", split = 'test')
dataset_validation = load_dataset("msislam/marc-multilingual-encodings-v4", split = 'validation')

In [9]:
dataset_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 60000
})

In [10]:
dataset_test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [11]:
dataset_validation

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [1]:
# creating the labels
label_names = ['I-DE', 'I-EN', 'I-ES', 'I-FR']

In [13]:
label_len = len(label_names)
label_len

4

In [15]:
# defining the model
model_name  = "xlm-roberta-base"

In [16]:
# defining the tokenizer
tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_name, add_prefix_space=True)

In [17]:
# defining data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
# defining the metric
metric = load_metric("seqeval")

<ipython-input-20-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [21]:
# creating id attaching the the labels and vice versa
id2label = {id : label for id, label in enumerate(label_names)}
label2id = {label: id for id, label in enumerate(label_names)}

In [22]:
id2label

{0: 'I-DE', 1: 'I-EN', 2: 'I-ES', 3: 'I-FR'}

In [23]:
label2id

{'I-DE': 0, 'I-EN': 1, 'I-ES': 2, 'I-FR': 3}

In [24]:
# function to calculate the metrics
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    print(all_metrics)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [25]:
# defining garbage collector
gc.collect()
torch.cuda.empty_cache()

In [26]:
# creating the model with GPU access
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=label_len,id2label=id2label,label2id=label2id).cuda()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to us

In [27]:
# model configuration
model.config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "I-DE",
    "1": "I-EN",
    "2": "I-ES",
    "3": "I-FR"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "I-DE": 0,
    "I-EN": 1,
    "I-ES": 2,
    "I-FR": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [28]:
# number of model parameters
model.num_parameters()

277456132

In [29]:
model.config.id2label

{0: 'I-DE', 1: 'I-EN', 2: 'I-ES', 3: 'I-FR'}

In [30]:
# setting up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    #save_strategy= "epoch",
    save_strategy = "steps",
    save_steps = 2500,
    evaluation_strategy="steps",
    eval_steps = 2500,
    logging_strategy = "steps",
    logging_steps = 2500,
    remove_unused_columns = False,
    #evaluation_strategy="epoch",
    learning_rate= 2e-5,
    auto_find_batch_size = True,
    #per_device_train_batch_size = 4,
    #per_device_eval_batch_size = 4,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none",
    fp16=True,
    #use_mps_device = True,
    metric_for_best_model="f1",
    load_best_model_at_end=True
)

In [31]:
dataset_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 60000
})

In [32]:
# setting up the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #callbacks=[log_history_callback]
)

In [33]:
# initiating the training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
2500,0.018700,0.005186,0.978588,0.978614,0.978601,0.998653
5000,0.005500,0.003667,0.979501,0.981860,0.980679,0.998867
7500,0.004600,0.005106,0.977785,0.981624,0.979701,0.998938
10000,0.003300,0.003168,0.984362,0.985367,0.984865,0.999139
12500,0.002900,0.003504,0.982344,0.984556,0.983449,0.998940
15000,0.003300,0.002936,0.987096,0.987200,0.987148,0.999256
17500,0.002100,0.002834,0.986009,0.986990,0.986500,0.999206
20000,0.003500,0.002912,0.987001,0.987828,0.987414,0.999266
22500,0.002200,0.002848,0.988217,0.987933,0.988075,0.999280
25000,0.001400,0.002810,0.987780,0.988168,0.987974,0.999332


{'DE': {'precision': 0.9795013850415513, 'recall': 0.9772455089820359, 'f1': 0.9783721466451465, 'number': 10855}, 'EN': {'precision': 0.9828953655040612, 'recall': 0.9834592217229181, 'f1': 0.9831772127700248, 'number': 10459}, 'ES': {'precision': 0.9744017144898203, 'recall': 0.981883623275345, 'f1': 0.9781283614198638, 'number': 8335}, 'FR': {'precision': 0.9762604301328006, 'recall': 0.9712381620484041, 'f1': 0.9737428203024265, 'number': 8553}, 'overall_precision': 0.9785880689998168, 'overall_recall': 0.9786136851473745, 'overall_f1': 0.9786008769059615, 'overall_accuracy': 0.9986532346100907}
{'DE': {'precision': 0.974217818148454, 'recall': 0.9781667434362045, 'f1': 0.9761882872115473, 'number': 10855}, 'EN': {'precision': 0.9830152671755725, 'recall': 0.9849890046849603, 'f1': 0.9840011461865419, 'number': 10459}, 'ES': {'precision': 0.9844106007914618, 'recall': 0.984883023395321, 'f1': 0.9846467554276119, 'number': 8335}, 'FR': {'precision': 0.9771455223880597, 'recall': 0.9

TrainOutput(global_step=37500, training_loss=0.0035408822377522788, metrics={'train_runtime': 12132.8604, 'train_samples_per_second': 24.726, 'train_steps_per_second': 3.091, 'total_flos': 4.531428232585574e+16, 'train_loss': 0.0035408822377522788, 'epoch': 5.0})

In [34]:
trainer.state.best_model_checkpoint

'./results/checkpoint-37500'

In [39]:
# saving the best model
trainer.save_model('results/checkpoint-best/')

In [41]:
# loading the training arguments of the best model
import pickle
with open("results/checkpoint-best/training_args.pkl", 'wb') as f:
    pickle.dump(obj=trainer.args, file=f)

In [42]:
trainer.args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=True,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=2500,
evaluation_strategy=steps,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=

In [45]:
# loading training state of the best model
with open('results/checkpoint-best/training_history.pkl', 'wb') as f:
    pickle.dump(obj=trainer.state, file=f)

In [43]:
trainer.state

TrainerState(epoch=5.0, global_step=37500, max_steps=37500, num_train_epochs=5, total_flos=4.531428232585574e+16, log_history=[{'loss': 0.0187, 'learning_rate': 1.866826666666667e-05, 'epoch': 0.33, 'step': 2500}, {'eval_loss': 0.005186041351407766, 'eval_precision': 0.9785880689998168, 'eval_recall': 0.9786136851473745, 'eval_f1': 0.9786008769059615, 'eval_accuracy': 0.9986532346100907, 'eval_runtime': 57.7748, 'eval_samples_per_second': 86.543, 'eval_steps_per_second': 10.818, 'epoch': 0.33, 'step': 2500}, {'loss': 0.0055, 'learning_rate': 1.7334933333333336e-05, 'epoch': 0.67, 'step': 5000}, {'eval_loss': 0.0036667557433247566, 'eval_precision': 0.9795007050712905, 'eval_recall': 0.9818595885032197, 'eval_f1': 0.980678728299519, 'eval_accuracy': 0.9988668185485785, 'eval_runtime': 57.4715, 'eval_samples_per_second': 87.0, 'eval_steps_per_second': 10.875, 'epoch': 0.67, 'step': 5000}, {'loss': 0.0046, 'learning_rate': 1.6003733333333334e-05, 'epoch': 1.0, 'step': 7500}, {'eval_loss':

In [44]:
trainer.state.log_history

[{'loss': 0.0187,
  'learning_rate': 1.866826666666667e-05,
  'epoch': 0.33,
  'step': 2500},
 {'eval_loss': 0.005186041351407766,
  'eval_precision': 0.9785880689998168,
  'eval_recall': 0.9786136851473745,
  'eval_f1': 0.9786008769059615,
  'eval_accuracy': 0.9986532346100907,
  'eval_runtime': 57.7748,
  'eval_samples_per_second': 86.543,
  'eval_steps_per_second': 10.818,
  'epoch': 0.33,
  'step': 2500},
 {'loss': 0.0055,
  'learning_rate': 1.7334933333333336e-05,
  'epoch': 0.67,
  'step': 5000},
 {'eval_loss': 0.0036667557433247566,
  'eval_precision': 0.9795007050712905,
  'eval_recall': 0.9818595885032197,
  'eval_f1': 0.980678728299519,
  'eval_accuracy': 0.9988668185485785,
  'eval_runtime': 57.4715,
  'eval_samples_per_second': 87.0,
  'eval_steps_per_second': 10.875,
  'epoch': 0.67,
  'step': 5000},
 {'loss': 0.0046,
  'learning_rate': 1.6003733333333334e-05,
  'epoch': 1.0,
  'step': 7500},
 {'eval_loss': 0.005106177646666765,
  'eval_precision': 0.977784730913642,
  'ev

In [46]:
!zip -r /content/file-all4-sk.zip /content/results/checkpoint-best

  adding: content/results/checkpoint-best/ (stored 0%)
  adding: content/results/checkpoint-best/special_tokens_map.json (deflated 52%)
  adding: content/results/checkpoint-best/tokenizer_config.json (deflated 50%)
  adding: content/results/checkpoint-best/training_args.pkl (deflated 46%)
  adding: content/results/checkpoint-best/pytorch_model.bin (deflated 22%)
  adding: content/results/checkpoint-best/config.json (deflated 52%)
  adding: content/results/checkpoint-best/training_args.bin (deflated 48%)
  adding: content/results/checkpoint-best/training_history.pkl (deflated 60%)
  adding: content/results/checkpoint-best/tokenizer.json (deflated 76%)


In [47]:
# evaluating the model
eval_results = trainer.evaluate()

{'DE': {'precision': 0.988663594470046, 'recall': 0.9882081989866421, 'f1': 0.9884358442755126, 'number': 10855}, 'EN': {'precision': 0.9914113942170054, 'recall': 0.9933071995410652, 'f1': 0.9923583914413985, 'number': 10459}, 'ES': {'precision': 0.9915956297274583, 'recall': 0.990881823635273, 'f1': 0.9912385981757081, 'number': 8335}, 'FR': {'precision': 0.9871375116931712, 'recall': 0.9870220975096458, 'f1': 0.9870798012277112, 'number': 8553}, 'overall_precision': 0.9897147343627323, 'overall_recall': 0.9899219936128999, 'overall_f1': 0.9898183531382506, 'overall_accuracy': 0.999424509943519}


In [48]:
eval_results

{'eval_loss': 0.002769122365862131,
 'eval_precision': 0.9897147343627323,
 'eval_recall': 0.9899219936128999,
 'eval_f1': 0.9898183531382506,
 'eval_accuracy': 0.999424509943519,
 'eval_runtime': 58.7866,
 'eval_samples_per_second': 85.053,
 'eval_steps_per_second': 10.632,
 'epoch': 5.0}

In [49]:
dataset_test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [50]:
# model prediction using test data
predictions, labels, _ = trainer.predict(dataset_test)
predictions = np.argmax(predictions, axis=-1)

{'DE': {'precision': 0.9870741390453328, 'recall': 0.9883516686696866, 'f1': 0.9877124907612713, 'number': 10817}, 'EN': {'precision': 0.9901617633147289, 'recall': 0.9914748508098892, 'f1': 0.9908178720181748, 'number': 10557}, 'ES': {'precision': 0.9912407007439404, 'recall': 0.9912407007439404, 'f1': 0.9912407007439406, 'number': 8334}, 'FR': {'precision': 0.9872469872469872, 'recall': 0.9871314927468414, 'f1': 0.9871892366188945, 'number': 8548}, 'overall_precision': 0.9888723454274744, 'overall_recall': 0.9895702634880803, 'overall_f1': 0.9892211813585232, 'overall_accuracy': 0.9993651810717168}


In [51]:
# calculating and printing prediction metrics
true_labels = [
    [label_names[l] for l in label  if l != -100]
    for label in labels
]

true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label)  if l != -100]
    for prediction, label in zip(predictions, labels)
]

prediction_results = metric.compute(predictions=true_predictions, references=true_labels)
print(prediction_results)

{'DE': {'precision': 0.9870741390453328, 'recall': 0.9883516686696866, 'f1': 0.9877124907612713, 'number': 10817}, 'EN': {'precision': 0.9901617633147289, 'recall': 0.9914748508098892, 'f1': 0.9908178720181748, 'number': 10557}, 'ES': {'precision': 0.9912407007439404, 'recall': 0.9912407007439404, 'f1': 0.9912407007439406, 'number': 8334}, 'FR': {'precision': 0.9872469872469872, 'recall': 0.9871314927468414, 'f1': 0.9871892366188945, 'number': 8548}, 'overall_precision': 0.9888723454274744, 'overall_recall': 0.9895702634880803, 'overall_f1': 0.9892211813585232, 'overall_accuracy': 0.9993651810717168}


In [52]:
# calling the best model
model_best = AutoModelForTokenClassification.from_pretrained("results/checkpoint-best/")

In [53]:
model_best.config

XLMRobertaConfig {
  "_name_or_path": "results/checkpoint-best/",
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "I-DE",
    "1": "I-EN",
    "2": "I-ES",
    "3": "I-FR"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "I-DE": 0,
    "I-EN": 1,
    "I-ES": 2,
    "I-FR": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.30.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [54]:
# example text
text = 'Sie sind gut. How about you? Comment va ta mère? And what about you?Estoy aprendiendo español. Thanks.'

In [ ]:
# example text
text = '"Hello, how are you? Guten Tag!"'

In [55]:
# creating embedded inputs using the tokenizer
tokens = tokenizer(text, add_special_tokens= False, return_tensors="pt")

In [56]:
tokens

{'input_ids': tensor([[   727,   1276,   5768,      5,  11249,   1672,    398,     32,  16277,
            307,    308,  94683,     32,   3493,   2367,   1672,    398,     32,
         159970,     53, 109802,   4017,  36131,      5,  22362,      5]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [57]:
# predicting using the best model
with torch.no_grad():
  logits = model_best(**tokens).logits

labels_predicted = logits.argmax(-1)

In [58]:
logits

tensor([[[ 6.4227, -2.0906, -2.9082, -1.9542],
         [10.3800, -3.4752, -3.3619, -3.5705],
         [10.4305, -3.5540, -3.3986, -3.4291],
         [10.1828, -3.4653, -3.4368, -3.6833],
         [-2.8922,  4.9866, -2.7963, -0.4872],
         [-3.0611,  5.1412, -2.8119, -0.5075],
         [-3.2989,  5.1967, -2.9386, -0.3128],
         [-2.8974,  4.9982, -2.7464, -0.5783],
         [-3.4862, -3.8999, -2.3259, 10.5741],
         [-3.3861, -4.0742, -2.2032, 10.6098],
         [-3.3893, -3.8983, -2.2133, 10.5120],
         [-3.4170, -3.6037, -2.5408, 10.4603],
         [-3.5466, -3.3968, -2.1601, 10.0394],
         [-2.5143,  5.1698, -2.4301, -1.5254],
         [-2.7888,  5.2349, -2.4451, -1.1374],
         [-2.7949,  5.7437, -2.6815, -1.5899],
         [-2.9699,  5.4787, -2.6213, -1.2770],
         [-2.6242,  5.0385, -2.2669, -1.3377],
         [-3.7312, -2.2774, 10.4162, -3.0205],
         [-3.7219, -2.5271, 10.5287, -2.7254],
         [-3.7170, -2.5520, 10.8948, -3.1360],
         [-3.

In [59]:
# predicted labels
labels_predicted

tensor([[0, 0, 0, 0, 1, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
         1, 1]])

In [60]:
# predicted language labels
lang_tag_predicted = [model_best.config.id2label[t.item()] for t in labels_predicted[0]]
lang_tag_predicted

['I-DE',
 'I-DE',
 'I-DE',
 'I-DE',
 'I-EN',
 'I-EN',
 'I-EN',
 'I-EN',
 'I-FR',
 'I-FR',
 'I-FR',
 'I-FR',
 'I-FR',
 'I-EN',
 'I-EN',
 'I-EN',
 'I-EN',
 'I-EN',
 'I-ES',
 'I-ES',
 'I-ES',
 'I-ES',
 'I-ES',
 'I-ES',
 'I-EN',
 'I-EN']

In [61]:
# loading HuggingFace library to push the model
from huggingface_hub import notebook_login
notebook_login()

In [66]:
# pushing the model to HuggingFace
model_best.push_to_hub("msislam/multilingual-language-detection-XLMRoberta", use_temp_dir=True )

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/msislam/multilingual-language-detection-XLMRoberta/commit/8797907a5307222e94f243704178506dd7237695', commit_message='Upload XLMRobertaForTokenClassification', commit_description='', oid='8797907a5307222e94f243704178506dd7237695', pr_url=None, pr_revision=None, pr_num=None)

In [67]:
# pushing the tokenizer to HuggingFace
tokenizer.push_to_hub("msislam/multilingual-language-detection-XLMRoberta", use_temp_dir=True )

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/msislam/multilingual-language-detection-XLMRoberta/commit/ff09bd9242d66ee7a437ee83d9a093d3bb8b8c54', commit_message='Upload tokenizer', commit_description='', oid='ff09bd9242d66ee7a437ee83d9a093d3bb8b8c54', pr_url=None, pr_revision=None, pr_num=None)